---
title: "Multiagent"
ico

---

# Multiagent

# Using agents as tools (supervisor)

<Tabs>
  <Tab title="Supervisor (from scratch)">

In [ ]:
from langchain.chat_models import init_chat_model
from langchain_core.tools import tool
from langgraph.prebuilt import create_react_agent

chat_model = init_chat_model("anthropic:claude-opus-4-20250514")

def book_hotel(hotel_name: str):
    """Book a hotel"""
    return f"Successfully booked a stay at {hotel_name}."

def book_flight(from_airport: str, to_airport: str):
    """Book a flight"""
    return f"Successfully booked a flight from {from_airport} to {to_airport}."

flight_assistant = create_react_agent(
    model="anthropic:claude-opus-4-20250514",
    tools=[book_flight],
    prompt="You are a flight booking assistant",
    name="flight_assistant"
)

hotel_assistant = create_react_agent(
    model="anthropic:claude-opus-4-20250514",
    tools=[book_hotel],
    prompt="You are a hotel booking assistant",
    name="hotel_assistant"
)


@tool
def booking_agent(instructions: str):
    """Give instructions to the booking agent about what flight to book."""
    results = flight_assistant.invoke({
        "messages": [{
        "role": "user",
        "content": instructions
        }]
    })
    return results['messages'][-1].content


@tool
def hotel_agent(instructions: str):
    """Give instructions to the hotel assistant about what hotel to book."""
    result = hotel_assistant.invoke({
        "messages": [{
        "role": "user",
        "content": instructions
        }]
    })
    return result['messages'][-1].content

supervisor = create_react_agent(
    model=chat_model,
    prompt=(
         "You manage a hotel booking assistant and a"
         "flight booking assistant. Assign work to them based on the user requests."
     ),
    tools=[hotel_agent, booking_agent],
)

response = supervisor.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "book a flight from BOS to JFK and a stay at McKittrick Hotel"
            }
        ]
    }
)


  </Tab>
  <Tab title="Supervisor prebuilt" icon="leaf">

In [ ]:
from langgraph.prebuilt import create_react_agent
from langgraph_supervisor import create_supervisor
from langchain.chat_models import init_chat_model

chat_model = init_chat_model("google_vertexai: gemini-2.5-flash")

def book_hotel(hotel_name: str):
    """Book a hotel"""
    return f"Successfully booked a stay at {hotel_name}."

def book_flight(from_airport: str, to_airport: str):
    """Book a flight"""
    return f"Successfully booked a flight from {from_airport} to {to_airport}."

flight_assistant = create_react_agent(
    model="anthropic:claude-opus-4-20250514",
    tools=[book_flight],
    prompt="You are a flight booking assistant",
    name="flight_assistant"
)

hotel_assistant = create_react_agent(
    model="anthropic:claude-opus-4-20250514",
    tools=[book_hotel],
    prompt="You are a hotel booking assistant",
    name="hotel_assistant"
)


hotel_assistant = create_react_agent(
    model="anthropic:claude-opus-4-20250514",
    tools=[book_hotel],
    prompt="You are a hotel booking assistant",
    name="hotel_assistant"
)

supervisor = create_supervisor(
    model=chat_model,
    prompt=(
        "You manage a hotel booking assistant and a"
        "flight booking assistant. Assign work to them."
    ),
   agents=[flight_assistant, hotel_assistant],
).compile()

response = supervisor.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "book a flight from BOS to JFK and a stay at McKittrick Hotel"
            }
        ]
    }
)

    </Tab>
</Tabs>